In [2]:
import numpy as np
import pandas as pd
import datetime
import copy
import time
import os
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import operator

from tqdm.auto import tqdm, trange
from tqdm.notebook import tqdm
from datetime import timedelta

tqdm.pandas()

In [3]:
chartevents_df = pd.read_parquet("data/CHARTEVENTS.parquet")
chartevents_df

,row_id,subject_id,hadm_id,icustay_id,itemid,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,788,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15,15.00,L/min,0.0,0.0,None,None
1,789,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100,100.00,None,0.0,0.0,None,None
2,790,36,165660,241249.0,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823.0,.37,0.37,None,0.0,0.0,None,None
3,791,36,165660,241249.0,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,6,6.00,min,0.0,0.0,None,None
4,792,36,165660,241249.0,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,2.5,2.50,None,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330712478,330471885,99781,147562,200664.0,224847,2133-08-02 08:30:00,2133-08-02 08:30:00,14357.0,Moderate,NaN,None,0.0,0.0,None,None
330712479,330471886,99781,147562,200664.0,224889,2133-08-02 08:30:00,2133-08-02 08:31:00,14357.0,Not applicable,NaN,None,0.0,0.0,None,None
330712480,330471887,99781,147562,200664.0,224903,2133-08-02 08:30:00,2133-08-02 08:31:00,14357.0,None,NaN,None,0.0,0.0,None,None
330712481,330471888,99781,147562,200664.0,224910,2133-08-02 08:30:00,2133-08-02 08:31:00,14357.0,None,NaN,None,0.0,0.0,None,None


`CHARTEVENTS` contains all the charted data available for a patient. During their ICU stay, the primary repository of a patient’s information is their electronic chart. The electronic chart displays patients’ routine vital signs and any additional information relevant to their care: ventilator settings, laboratory values, code status, mental status, and so on. 

Some data maybe frequently repeated within `CHARTEVENTS`, as some data may be desireable to display on a patient's electronic chart. (ex. `LABEVENTS`)

In [4]:
print(f"There are {chartevents_df.subject_id.nunique()} patients who have chart data")

There are 46467 patients who have chart data


`itemid` is categorical data. It refers to the type of measurement taken. Each row is associated with one `itemid`, corresponds to an instantiation of the same measurement.

Refer to `D_ITEMS` table for exact definitions of each `itemid`.

In [5]:
c = chartevents_df.itemid.value_counts()[:5]
p = chartevents_df.itemid.value_counts(normalize=True).mul(100).round(2)[:5]
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
211,5180809,1.57
742,3464326,1.05
646,3418917,1.03
618,3386719,1.02
212,3303151,1.00


`charttime` refers to when the observation was made, and `storetime` records the time when the observation was manually input or validated by clinical staff.

`cgid` is the identifier for the caregivers who validated the measurement, this is directly related to the `caregivers.parquet` table.

In [6]:
c = chartevents_df.cgid.value_counts().nlargest(5)
p = chartevents_df.cgid.value_counts(normalize=True).mul(100).round(2).nlargest(5)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
21570.0,12923614,3.98
15331.0,8952525,2.76
20889.0,4624264,1.43
19611.0,1629962,0.50
16302.0,1362164,0.42


`value` and `valuenum` correspond to the value measured for `itemid`. If `value` is numeric, then `value` and `valuenum` are the exact same. If `value` is not numeric, then `valuenum` will be NULL. `valueuom` is the unit of measurement for `value`, if applicable. 

`warning` and `error` are binary valued columns, which specify if a warning or an error value occured for that measurement.

In [7]:
c = chartevents_df.warning.value_counts()
p = chartevents_df.warning.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
0.0,107161412,97.96
1.0,2235230,2.04


In [8]:
c = chartevents_df.error.value_counts()
p = chartevents_df.error.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
0.0,109263322,99.88
1.0,133320,0.12


`resultstatus` and `stopped` specify the type of measurement (RESULTSTATUS is ‘Manual’ or ‘Automatic’) and whether the measurement was stopped.

In [11]:
c = chartevents_df.resultstatus.value_counts()
p = chartevents_df.resultstatus.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
Final,8827334,92.63
Manual,702326,7.37


In [12]:
c = chartevents_df.stopped.value_counts()
p = chartevents_df.stopped.value_counts(normalize=True).mul(100).round(2)
pd.concat([c,p], axis=1, keys=['counts', '%'])

,counts,%
NotStopd,219536113,99.59
D/C'd,894085,0.41
